# Предобработка данных

In [36]:
import mysql.connector
import pandas as pd
import numpy as np

<p>Поскольку данные находятся в <b>таблицах в базе данных</b>, прежде всего их необходимо <b>загрузить в датафреймы</b> для будущей работы с ними.</p>

In [2]:
# database connection establishment

con = mysql.connector.connect(host='localhost', user='root', password='Pahomov2000+', db='nutritionaldb')

In [3]:
# extracting data from tables into dataframes

df_complexes = pd.read_sql("SELECT * FROM complexes", con)
df_employees = pd.read_sql("SELECT * FROM employees", con)
df_dishes = pd.read_sql("SELECT * FROM dishes", con)
df_employeeselect = pd.read_sql("SELECT * FROM employeeselect", con)
df_dishesincomplex = pd.read_sql("SELECT * FROM dishesincomplex", con)

<h2><b>Почему необходимо введение столбца "Category"?</b></h2>
<p>Это необходимо, поскольку в данных есть много "блюд-синонимов".</p>
<p>Например: <b>Рассольник</b> и <b>Рассольник Ленинградский</b> и т. п.</p>
<p>Если не принять данную меру, то при работе с ML моделью мы будем получать неправильные предсказания.</p>
<p>Также с помощью <i>категоризации</i> мы частично решаем проблему уменьшения размерности пространства признаков.</p>
<p> </p>

In [4]:
# Preparing new column and fill it with 'no'

df_dishes['Category'] = 'no'

In [5]:
# fill_cat fills category column value with its Name

def fill_cat(df, name):
    df.loc[df['Name'].str.contains(name), ['Category']] = name

In [6]:
# change_cat changes a wrong category name to a new correct

def change_cat(df, name, new_name):
    df.loc[df['Name'].str.contains(name), ['Category']] = new_name

In [7]:
# get unique sorted dishes names and fit category column with it

unique_names = []

for name in df_dishes['Name']:
    unique_names.append(name.split(' ')[0])

unique_names = sorted(set(unique_names))

for cat in unique_names:
    fill_cat(df_dishes, cat)

In [8]:
df_dishes.head(3)

,Id,Name,Category
0,1,Азу по-татарски,Азу
1,2,Бигос,Бигос
2,3,Биточек куриный с яйцом и луком,Биточек


<p> </p>
<p>Для ускорения процесса установки блюду его категории применялась функция, которая определяла категорию, как первое слово названия блюда. И действительно, такая тактика подошла большей части блюд. Но есть и исключения, которые надо обработать.</p> <p>Такой метод <i>категоризации</i> не сработает на блюдах:</p> 
<ul>
    <li>"Картофель запеченый", "Картофель жареный"...</li> 
    <li>"Каша гречневая", "Каша овсяная"...</li>
    <li>"Салат оливье", "Салат витаминный"...</li>
    <li>и т. д.</li>
</ul>
<p>поскольку у этих <i>вроде бы похожих блюд</i> абсолютно разные способы приготовления и вкус. Соответственно, для таких исключений в столбец <b>Category</b> идёт название блюда.</p>

In [10]:
# Id 1 - 60

change_cat(df_dishes, 'Биточки', 'Биточек')
change_cat(df_dishes, 'Гороховое', 'Гороховое пюре')
change_cat(df_dishes, 'Гороховый', 'Гороховый суп')

change_cat(df_dishes, 'Картофель жареный', 'Картофель жареный')
change_cat(df_dishes, 'Картофель запеченый', 'Картофель запеченый')
change_cat(df_dishes, 'Картофель запеченый', 'Картофель запеченый')
change_cat(df_dishes, 'Картофель отварной', 'Картофель отварной')
change_cat(df_dishes, 'Картофель тушеный в сметаном соусе', 'Картофель тушеный в сметаном соусе')
change_cat(df_dishes, 'Картофель по-селянски', 'Картофель по-селянски')
change_cat(df_dishes, 'Картофельное пюре', 'Картофельное пюре')

change_cat(df_dishes, 'Каша гречневая', 'Каша гречневая')
change_cat(df_dishes, 'Каша гречневая с мясом', 'Каша гречневая с мясом')


# Id 61 - 120

change_cat(df_dishes, 'Каша овсяная', 'Каша овсяная')
change_cat(df_dishes, 'Каша перловая', 'Каша перловая')
change_cat(df_dishes, 'Каша пшеничная', 'Каша пшеничная')
change_cat(df_dishes, 'Каша пшеничная', 'Каша пшеничная')
change_cat(df_dishes, 'Каша пшенная', 'Каша пшенная')
change_cat(df_dishes, 'Каша рисовая', 'Каша рисовая')
change_cat(df_dishes, 'Каша ячневая', 'Каша ячневая')
change_cat(df_dishes, 'Котлета из курицы', 'Котлета куриная')
change_cat(df_dishes, 'Котлета куриная', 'Котлета куриная')
change_cat(df_dishes, 'Котлета рыбная', 'Котлета рыбная')
change_cat(df_dishes, 'Макароны по-флотски', 'Макароны по-флотски')
change_cat(df_dishes, 'Мясное суфле', 'Мясное суфле')


# Id 121 - 180

change_cat(df_dishes, 'Похлебка по-суворовски', 'Похлебка по-суворовски')

# Salads processing
df_dishes.loc[df_dishes['Name'].str.contains('Сал.'), ['Category']] = df_dishes[df_dishes['Name'].str.contains('Сал.')].Name.values
df_dishes.loc[df_dishes['Name'].str.contains('морков'), ['Category']] = 'Салат из моркови'
df_dishes.loc[df_dishes['Name'].str.contains('Морков') & (df_dishes['Id'] > 190), ['Category']] = 'Салат из моркови'


In [11]:
# Extracting actual names with prefix "Сал. " into array
# КОСТЫЛЬ, переписать этот говнокод

arr = df_dishes.loc[df_dishes['Name'].str.contains('Сал. '), ['Name']]['Name']
sal_categories = []

for s in arr:
    cat_arr = s.split(" ")
    cat_arr[0] = 'Салат'
    sal_categories.append(" ".join(cat_arr))

In [13]:
# Values with "Сал. " in Category processing
# Id 138 - 145

df_dishes.loc[df_dishes['Category'].str.contains('Сал. '), ['Category']] = sal_categories

# Soup processing

change_cat(df_dishes, 'Суп борщ', 'Суп борщ')
change_cat(df_dishes, 'Суп гороховый', 'Суп гороховый')
change_cat(df_dishes, 'Суп куриный', 'Суп куриный')
change_cat(df_dishes, 'Суп овощной', 'Суп овощной')
change_cat(df_dishes, 'Суп рассольник ленинградский', 'Рассольник')
change_cat(df_dishes, 'Суп картоф', 'Суп картофельный')
change_cat(df_dishes, 'Суп с крупой', 'Суп с крупой')
change_cat(df_dishes, 'Суп тыкмач', 'Суп тыкмач')
change_cat(df_dishes, 'Сырный суп', 'Сырный суп')
change_cat(df_dishes, 'Суп харчо', 'Харчо')

change_cat(df_dishes, 'Тушен', 'Тушеные куриные желудки')
change_cat(df_dishes, 'Хлеб 2 куска', 'Хлеб')
change_cat(df_dishes, 'Шарик куриный с сыром', 'Шарик куриный с сыром')


# Correcting similar dishes title to common

df_dishes.loc[df_dishes['Name'].str.contains('краб') | df_dishes['Name'].str.contains('Краб'), ['Category']] = 'Салат крабовый'
df_dishes.loc[df_dishes['Name'].str.contains('свек') | df_dishes['Name'].str.contains('Свек'), ['Category']] = 'Салат свекольный'
df_dishes.loc[df_dishes['Name'].str.contains('морской капусты') | df_dishes['Name'].str.contains('Морской капусты'), ['Category']] = 'Салат из морской капусты'
df_dishes.loc[df_dishes['Name'].str.contains('помид') & df_dishes['Name'].str.contains('огур'), ['Category']] = 'Салат из огурцов и помидоров'

In [14]:
# Creating dataframe df_categories where I'm mapping Category and Category ID

categories = sorted(set(df_dishes['Category']))
cat_id = np.arange(1, len(categories) + 1)

df_categories = pd.DataFrame({'Id' : cat_id, 'Category' : categories})

In [16]:
df_categories.head()

,Id,Category
0,1,Азу
1,2,Бигос
2,3,Биточек
3,4,Бифштекс
4,5,Борщ


In [19]:
# Prepare data for Category ID column

cat_ids = []

for cat in df_dishes['Category']:
    cat_ids.append(df_categories[df_categories['Category'] == cat].Id.values[0])

# Insert prepared data into Category ID

df_dishes['Cat_Id'] = cat_ids

In [21]:
df_dishes.head()

,Id,Name,Category,Cat_Id
0,1,Азу по-татарски,Азу,1
1,2,Бигос,Бигос,2
2,3,Биточек куриный с яйцом и луком,Биточек,3
3,4,Биточек мясной с яйцом и луком,Биточек,3
4,5,Биточки паровые,Биточек,3


<p> </p>
<p>Если обратить внимание, то можно заметить, что среди блюд в комплексах есть <i>"блюда"</i>, которые <b>абсолютно никак не влияют на выбор сотрудника</b>.</p>
<p>Такими компонентами являются:</p>
<ul>
    <li>Компот</li> 
    <li>Майонез</li>
    <li>Набор</li>
    <li>Подлива</li>
    <li>Сметана</li> 
    <li>Соус</li>
    <li>Сыр</li>
    <li>Хлеб</li>
    <li>Чай</li>
</ul>
<p>Соответственно, из датафреймов, в которых <i>"мелькают"</i> данные <i>"блюда"</i> необходимо <b>удалить</b> соответствующие строчки.</p>

In [22]:
FORBIDDEN_CATEGORIES = ['Компот', 'Майонез', 'Набор', 'Подлива', 'Сметана', 'Соус', 'Сыр', 'Хлеб', 'Чай']

In [23]:
FORBIDDEN_DISHES_ID = []

for cat in FORBIDDEN_CATEGORIES:
    FORBIDDEN_DISHES_ID += list(df_dishes[df_dishes['Category'] == cat].Id.values)

In [24]:
FORBIDDEN_CATEGORIES_ID = []

for cat in FORBIDDEN_CATEGORIES:
    FORBIDDEN_CATEGORIES_ID += list(df_categories[df_categories['Category'] == cat].Id.values)

In [26]:
# returns list of lines in dataframe to drop

def get_lines(df, ids, col):
    lines = []
    
    for id_ in ids:
        lines += list(df[df[col] == id_].index)
        
    return lines

In [27]:
ids_to_remove = get_lines(df_dishesincomplex, FORBIDDEN_DISHES_ID, col='DishId')

In [28]:
dishes_remove = get_lines(df_dishes, FORBIDDEN_DISHES_ID, col='Id')

In [29]:
categories_remove = get_lines(df_categories, FORBIDDEN_CATEGORIES_ID, col='Id')

In [30]:
df_dishesincomplex.drop(ids_to_remove, inplace=True)
df_dishes.drop(dishes_remove, inplace=True)
df_categories.drop(categories_remove, inplace=True)

In [31]:
# returns max number of dishes in a complex in the whole orders history

def max_dishes(df):
    
    lengths = []
    dates = set(df['Date'])
    
    for date in dates:
        for cmp in range(3):
            lengths.append(df[(df['Date'] == date) & (df['CmpxId'] == cmp + 1)].shape[0])
            
    return max(lengths)

In [32]:
# transpose tensor in a special way...

def transpose(mat):
    new_mat = []
    
    for i in range(mat.shape[1]):
        dim2 = []
        
        for j in range(mat.shape[2]):
            dim1 = []
            
            for k in range(mat.shape[0]):
                dim1.append(mat[k][i][j])
            
                
            dim2.append(dim1)
            
        new_mat.append(dim2)
    
    return new_mat

In [33]:
# this function returns result non-encoded table of ranged data

def create_result_table(dishes, employees, dish_in_cmp, selected):
    dates = sorted(set(selected['Date']))
    
    MAX_DISHES = max_dishes(dish_in_cmp)
    NUM_OF_CMPX = 3
    
    res_dates = []
    res_emp_id = []
    res_dishes = []
    res_cmpx_id = []
    
    counter = 0
    
    print('RUNNING...\n')
    
    for date in dates:
        
        cmp_for_date = []
        
        for cmp in range(NUM_OF_CMPX):
            tmp_complex = []

            for dish in dish_in_cmp[(dish_in_cmp['Date'] == date) & (dish_in_cmp['CmpxId'] == cmp+1)].DishId:
                
                if dishes[dishes['Id'] == dish].Cat_Id.sum() != 0:
                    tmp_complex.append(dishes[dishes['Id'] == dish].Cat_Id.values[0])
                    
                else:
                    tmp_complex.append(0)
            
            if len(tmp_complex) < MAX_DISHES:
                while len(tmp_complex) != MAX_DISHES:
                    tmp_complex.append(0)
            
            tmp_complex = sorted(tmp_complex)
            
            cmp_for_date.append(tmp_complex)
            
        
        for emp in selected[selected['Date'] == date].EmpId:
            
            res_dates.append(date)
            res_emp_id.append(emp)
            res_cmpx_id.append(selected[(selected['Date'] == date) & (selected['EmpId'] == emp)].CmpxId.values[0])
            res_dishes.append(cmp_for_date)
        
        counter += 1
        
    
    vocab = {'EmpId' : res_emp_id, 'Date' : res_dates, 'CmpxId' : res_cmpx_id}
    
    res_dishes = transpose(np.array(res_dishes))
    
    for cmp in range(NUM_OF_CMPX):
        for d in range(MAX_DISHES):
            vocab['Dish_' + str(cmp+1) + str(d+1)] = res_dishes[cmp][d]
    
    result = pd.DataFrame(vocab)
    
    print('Processed all ' + str(counter) + ' dates')
    
    return result

<p> </p>
<p>Когда данные более-менее готовы, их необходимо <b>собрать в одну таблицу</b>, с которой в будущем будет работать <i>ML-модель</i>.</p>

In [34]:
# get result table

table = create_result_table(df_dishes, df_employees, df_dishesincomplex, df_employeeselect)

RUNNING...

Processed all 379 dates


In [35]:
table.head()

,EmpId,Date,CmpxId,Dish_11,Dish_12,Dish_13,Dish_14,Dish_15,Dish_16,Dish_21,...,Dish_23,Dish_24,Dish_25,Dish_26,Dish_31,Dish_32,Dish_33,Dish_34,Dish_35,Dish_36
0,9,2019-03-01,2,0,0,41,67,130,152,0,...,0,10,90,144,0,0,0,4,24,152
1,17,2019-03-01,2,0,0,41,67,130,152,0,...,0,10,90,144,0,0,0,4,24,152
2,28,2019-03-01,1,0,0,41,67,130,152,0,...,0,10,90,144,0,0,0,4,24,152
3,33,2019-03-01,2,0,0,41,67,130,152,0,...,0,10,90,144,0,0,0,4,24,152
4,35,2019-03-01,3,0,0,41,67,130,152,0,...,0,10,90,144,0,0,0,4,24,152


In [85]:
# storing table in .csv file

table.to_csv("E:\\Data Science\\Datasets\\EatingData\\result_table.csv", index=False)